In [1]:
import networkx as nx # version 2.2
import matplotlib.pyplot as plt
import re
import cvxpy as cp
import operator #to sort elements in a list of tuples
import itertools
import math
import numpy as np
import os
import sys
import time
import random

import Init_NetRate as Init
import cvxpy as cp
import Cascade_generation_functions_NetRate as Gen
import CVX_functions as cvx
import NetRate_ADMM_fcts as ADMM
import NetRate_SG_fcts as NetSG

import Init_NetInf
import Greedy_NetInf as Greed

In [2]:
def Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat,lower_bound):
    correct = 0
    nb_edges = 0
    top_acc = 0
    bot_acc = 0
    for i in range(0,G_true.number_of_nodes()):
        for j in range(0,G_true.number_of_nodes()):
            if A_hat[i,j] >lower_bound:
                nb_edges +=1
                bot_acc +=1
                if (i,j) in G_true.edges():
                    correct +=1
                    bot_acc+=1
                else :
                    top_acc+=1
            else :
                if (i,j) in G_true.edges():
                    top_acc +=1
                    bot_acc+=1
                
    print("Precision :",correct/nb_edges)
    print("Recall :",correct/G_true.number_of_edges())
    Accuracy = 1-(top_acc/bot_acc)
    print("Accuracy is ",Accuracy)
    mse = 0
    nb_correcte_edges=0
    for edge in G_true.edges():
        if A_hat[edge]>eps:
            nb_correcte_edges +=1
            try:
                true_alpha = G_true.edges[edge]["weight"][0]
            except TypeError:
                true_alpha = G_true.edges[edge]["weight"]
            approx_alpha = A_hat[edge[0],edge[1]]
            mse += pow((true_alpha-approx_alpha),2) # mean square error
    #         print("For edge " + str(edge)+ " the true weight is : " + str(true_alpha))
    #         print("For edge " + str(edge)+ " the approx weight is : " + str(approx_alpha))
    try :
        mse = mse/nb_correcte_edges
    except ZeroDivisionError:
        print ("There is no correct edge")
    print(" Mean square error is : ",mse)
def Global_Precision_Recall_Accuracy_NetInf(G_true,G_max):
    correct = 0
    top_acc = 0
    bot_acc = 0
    nb_edges_G_max = G_max.number_of_edges()
    nb_edges_G_true = G_true.number_of_edges()
    for i in range(0,G_true.number_of_nodes()):
        for j in range(0,G_true.number_of_nodes()):
            if (i,j) in G_max.edges() :
                bot_acc +=1
                if(i,j) in G_true.edges() :
                    bot_acc +=1
                    correct+=1
                else :
                    top_acc +=1
            else :
                if (i,j) in G_true.edges():
                    top_acc +=1
                    bot_acc +=1
    precision = correct/nb_edges_G_max
    recall = correct/nb_edges_G_true
    Accuracy = 1-top_acc/bot_acc
    print("precision is ",precision)
    print("Recall is ", recall)
    print("Accuracy is ",Accuracy)

In [3]:
network_file_name = "./Graph_test_SG.txt"
cascade_file_name = "./Cascade_test_SG.txt"
window = 10
model=0 # not important but needed. When constructing the underlying network specify that we use the exponential law
beta = 1 # used for the construction of the cascades, probability of the edge to transmit the infection
eps = 0.0005
alpha_max = 10 #bound alpha from above
alpha_init = 0.01 #initial value of the infectious rates
iter_ADMM = 50 #number of itteration for 1 node in the ADMM method. This is a parameter to tune
iter_GD = 500 #number of iterations of the gradient descent
gamma_ADMM = 0.00005 # Learning rate of the GD for alpha (maybe too small)
gamma_SGD = 0.0005
u = 4 # used for the gradient descent of rho and as a penalizer and the constrain
tol = 1e-2
K = 15000 #number of itterations of SG

EPS = 1e-64 #zero machine
ALPHA = 1.0 #Incubation parameter (for exp and power law)
MODEL = 0 # 0 = exp law, 1 = power law (power law is not fully implemented yet)
MAX = sys.float_info.max #Max value of a float in python
#MIN = sys.float_info.min #Min value of a float in python
MIN = -MAX

#(works only if groundtruth is available)
#When set to True (especially boundOn) it slow down greatly the computation
compare_groud_truth = False # If set to True outputs some aditional information (precision and recall of the algo)
boundOn = False


greedy_global_param = (ALPHA,MODEL,MAX,MIN,EPS,compare_groud_truth,boundOn)

In [11]:
# G_true = Init.Load_ground_truth(network_file_name)
# G_star, DAG_C_dic = Init.Init(cascade_file_name)
# N = G_true.number_of_nodes()
cascade_file = "Cascade_gen.txt"
graph_true_file ="test.txt"
G_true = Gen.Generate_random_graph(10,20)
Cascades = Gen.Generate_all_cascades(G_true,-100,window,model,beta)
Gen.Save_cascade_to_file(cascade_file,Cascades,G_true)
Gen.Save_graph_to_file(graph_true_file,G_true)
G_star,DAG_C_dic = Init.Init(cascade_file)
N = G_true.number_of_nodes()


G_star_NF,DAG_Tree_c_dic_NF,cascades_per_edge_dic_NF,edge_gain_dic_NF = Init_NetInf.Init(cascade_file,EPS,MAX)

All nodes were read
All nodes were read


In [6]:
nb_max_edge = int(1*G_true.number_of_edges()) #fix a number of edges we want to recover (here 90%)
ground_truth = G_true
t_s = time.time()
G_max,precision,recall,edge_info = Greed.GreedyOpt(nb_max_edge,DAG_Tree_c_dic_NF,cascades_per_edge_dic_NF,edge_gain_dic_NF,G_star_NF,True,greedy_global_param)
t_f = time.time()
Global_Precision_Recall_Accuracy_NetInf(G_true,G_max)
print ("Netinf took : ",t_f-t_s)

itteration :  0
itteration :  1
itteration :  2
itteration :  3
itteration :  4
itteration :  5
itteration :  6
itteration :  7
itteration :  8
itteration :  9
itteration :  10
itteration :  11
itteration :  12
itteration :  13
itteration :  14
itteration :  15
itteration :  16
itteration :  17
itteration :  18
itteration :  19
itteration :  20
itteration :  21
itteration :  22
itteration :  23
itteration :  24
itteration :  25
itteration :  26
itteration :  27
itteration :  28
itteration :  29
itteration :  30
itteration :  31
itteration :  32
itteration :  33
itteration :  34
itteration :  35
itteration :  36
itteration :  37
itteration :  38
itteration :  39
itteration :  40
itteration :  41
itteration :  42
itteration :  43
itteration :  44
itteration :  45
itteration :  46
itteration :  47
itteration :  48
itteration :  49
itteration :  50
itteration :  51
itteration :  52
itteration :  53
itteration :  54
itteration :  55
itteration :  56
itteration :  57
itteration :  58
itterat

itteration :  462
itteration :  463
itteration :  464
itteration :  465
itteration :  466
itteration :  467
itteration :  468
itteration :  469
itteration :  470
itteration :  471
itteration :  472
itteration :  473
itteration :  474
itteration :  475
itteration :  476
itteration :  477
itteration :  478
itteration :  479
itteration :  480
itteration :  481
itteration :  482
itteration :  483
itteration :  484
itteration :  485
itteration :  486
itteration :  487
itteration :  488
itteration :  489
itteration :  490
itteration :  491
itteration :  492
itteration :  493
itteration :  494
itteration :  495
itteration :  496
itteration :  497
itteration :  498
itteration :  499
itteration :  500
itteration :  501
itteration :  502
itteration :  503
itteration :  504
itteration :  505
itteration :  506
itteration :  507
itteration :  508
itteration :  509
itteration :  510
itteration :  511
itteration :  512
itteration :  513
itteration :  514
itteration :  515
itteration :  516
itteration

itteration :  919
itteration :  920
itteration :  921
itteration :  922
itteration :  923
itteration :  924
itteration :  925
itteration :  926
itteration :  927
itteration :  928
itteration :  929
itteration :  930
itteration :  931
itteration :  932
itteration :  933
itteration :  934
itteration :  935
itteration :  936
itteration :  937
itteration :  938
itteration :  939
itteration :  940
itteration :  941
itteration :  942
itteration :  943
itteration :  944
itteration :  945
itteration :  946
itteration :  947
itteration :  948
itteration :  949
itteration :  950
itteration :  951
itteration :  952
itteration :  953
itteration :  954
itteration :  955
itteration :  956
itteration :  957
itteration :  958
itteration :  959
itteration :  960
itteration :  961
itteration :  962
itteration :  963
itteration :  964
itteration :  965
itteration :  966
itteration :  967
itteration :  968
itteration :  969
itteration :  970
itteration :  971
itteration :  972
itteration :  973
itteration

itteration :  1356
itteration :  1357
itteration :  1358
itteration :  1359
itteration :  1360
itteration :  1361
itteration :  1362
itteration :  1363
itteration :  1364
itteration :  1365
itteration :  1366
itteration :  1367
itteration :  1368
itteration :  1369
itteration :  1370
itteration :  1371
itteration :  1372
itteration :  1373
itteration :  1374
itteration :  1375
itteration :  1376
itteration :  1377
itteration :  1378
itteration :  1379
itteration :  1380
itteration :  1381
itteration :  1382
itteration :  1383
itteration :  1384
itteration :  1385
itteration :  1386
itteration :  1387
itteration :  1388
itteration :  1389
itteration :  1390
itteration :  1391
itteration :  1392
itteration :  1393
itteration :  1394
itteration :  1395
itteration :  1396
itteration :  1397
itteration :  1398
itteration :  1399
itteration :  1400
itteration :  1401
itteration :  1402
itteration :  1403
itteration :  1404
itteration :  1405
itteration :  1406
itteration :  1407
itteration :

itteration :  1788
itteration :  1789
itteration :  1790
itteration :  1791
itteration :  1792
itteration :  1793
itteration :  1794
itteration :  1795
itteration :  1796
itteration :  1797
itteration :  1798
itteration :  1799
itteration :  1800
itteration :  1801
itteration :  1802
itteration :  1803
itteration :  1804
itteration :  1805
itteration :  1806
itteration :  1807
itteration :  1808
itteration :  1809
itteration :  1810
itteration :  1811
itteration :  1812
itteration :  1813
itteration :  1814
itteration :  1815
itteration :  1816
itteration :  1817
itteration :  1818
itteration :  1819
itteration :  1820
itteration :  1821
itteration :  1822
itteration :  1823
itteration :  1824
itteration :  1825
itteration :  1826
itteration :  1827
itteration :  1828
itteration :  1829
itteration :  1830
itteration :  1831
itteration :  1832
itteration :  1833
itteration :  1834
itteration :  1835
itteration :  1836
itteration :  1837
itteration :  1838
itteration :  1839
itteration :

In [5]:
A_list,nb_c_per_node = cvx.Create_matrices(G_true,DAG_C_dic,window)
t_s = time.time()
alpha_mat,final_obj_value = cvx.Infer_Network_edges(G_true,A_list,nb_c_per_node,DAG_C_dic)
t_f = time.time()
print("NetRate CVX took : ",t_f-t_s)
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,alpha_mat,0.05)
np.save("alpha_CVX",alpha_mat)

For node  0
optimal
-90.8904400800495
For node  1
optimal
-81.6482240171513
For node  2
optimal
-90.60600955685072
For node  3
optimal
-109.00220971140517
For node  4
optimal
-100.8602346464926
For node  5
optimal
-21.659304338175698
For node  6
optimal
-0.0
For node  7
optimal
-111.92082976164868
For node  8
optimal
-65.33003580490522
For node  9
optimal
-140.12148065000113
For node  10
optimal
-179.50591133432968
For node  11
optimal
-107.83935380473704
For node  12
optimal
-102.81243780975494
For node  13
optimal
-101.4501546509409
For node  14
optimal
-131.34366594381876
For node  15
optimal
-84.08719713004889
For node  16
optimal
-103.42405991043032
For node  17
optimal
-83.80443475143396
For node  18
optimal
-8.194520681261476
For node  19
optimal
-38.23240960994289
For node  20
optimal
-119.30038133274778
For node  21
optimal
-65.03334376927859
For node  22
optimal
-80.799874301233
For node  23
optimal
-59.28777169888582
For node  24
optimal
-121.16184075907204
For node  25
opti

In [18]:
t_s = time.time()
A_hat_SG = NetSG.Netrate_SG(DAG_C_dic,K,G_true,alpha_init,gamma_SGD,eps,window,N,alpha_max)
t_f = time.time()
print("NetRate SG took ",t_f-t_s)
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_SG,0.05)
np.save("alpha_SGD",A_hat_SG)

NetRate SG took  2.181178092956543
Precision : 1.0
Recall : 1.0
Accuracy is  1.0
 Mean square error is :  0.010492486881301034


In [28]:
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_SG,0.01)

Precision : 0.9523809523809523
Recall : 1.0
Accuracy is  0.975609756097561
 Mean square error is :  0.010492486881301034


In [32]:
G_Inferred = nx.DiGraph()
for i in range(0,N):
    G_Inferred.add_node(i)
for i in range(0,N):
    for j in range(0,N):
        if A_hat_SG[i,j]>=0.01 :
            G_Inferred.add_edge(i,j)
nx.write_gexf(G_Inferred,"G_inferred.gexf")

In [34]:
for edge in G_Inferred.edges():
    if edge not in G_true.edges():
        print(edge)

(7, 8)


In [12]:
t_s = time.time()
A_hat_ADMM,obj_per_iter_per_node_ADMM,recall_precision_ADMM = ADMM.NetRate_ADMM(G_true,N,alpha_init,DAG_C_dic,window,iter_GD,iter_ADMM,gamma_ADMM,alpha_max,u,eps,tol)
t_f = time.time()
print("ADMM took ",t_f-t_s)
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_ADMM,0.05)
np.save("alpha_ADMM",A_hat_ADMM)

ADMM took  1.586078405380249
Precision : 1.0
Recall : 1.0
Accuracy is  1.0
 Mean square error is :  0.011665898050518902


In [17]:
Global_Precision_Recall_Accuracy_MSE_NetRate(G_true,A_hat_ADMM,0)


Precision : 1.0
Recall : 1.0
Accuracy is  1.0
 Mean square error is :  0.011665898050518902


In [16]:
nx.write_gexf(G_true,"Test.gexf")